# Create Single DataFrame with all County Data

In [141]:
import pandas as pd
import glob
import re
from pathlib import Path

In [142]:
def grab_county_file_paths():
    p = Path("countyCSV_March7")
    paths = list(p.glob("*.csv"))
    return paths


In [143]:
county_file_paths = grab_county_file_paths()

In [144]:
# county_file_paths

In [145]:
def process_csvs(paths):
    valid_dfs = []
    invalid_dfs = []
    
    for path in paths:
        df = pd.read_csv(path)
        
        county = path.stem
        
        columns = df.columns
        if "0" in columns or "1" in columns:
            header = df.iloc[0]
            df = df.rename(header, axis="columns")
            df = df.iloc[1:]
        columns = df.columns
        
        if "Virginia" in columns[0] or "Va" in columns[0] and "Virginia" not in columns[1]:
            df = df.rename({columns[0]: "code_va", columns[1]: "local_code"}, axis=1)
            df["locality"] = county
            df = df.set_index("code_va")
            valid_dfs.append(df)
        else: 
            invalid_dfs.append(path)

    return valid_dfs, invalid_dfs
    

In [146]:
valid_dfs, invalid_dfs = process_csvs(county_file_paths)

In [147]:
len(valid_dfs)

84

In [148]:
len(invalid_dfs)

1

In [149]:
for p in invalid_dfs:
    df = pd.read_csv(p)
    print(df.head())

  Code of  Virgina  Section  salem
0              1-10 et seq.  Ch. 1
1                    1-13.3    1-2
2                  1-13.3:1    1-2
3            1-13.6, 1-13.7    1-2
4                    1-13.9    1-6


In [150]:
for df in valid_dfs[:3]:
    print(df.head())

                  local_code             locality
code_va                                          
1-3.9                    1-3  spotsylvania_county
1-13.3 et seq.           1-2  spotsylvania_county
2.1-116.1             16-147  spotsylvania_county
2.1-116.1 et seq.     16-147  spotsylvania_county
2.1-340 et seq.     2-107(b)  spotsylvania_county
              local_code      locality
code_va                               
1-1                  1-2  falls_church
1-21                 1-2  falls_church
1-200 et seq.        1-2  falls_church
1-210                1-2  falls_church
1-216                1-2  falls_church
        local_code  locality
code_va                     
1-210          1-2  richmond
1-216          1-2  richmond
1-217          1-8  richmond
1-222          1-2  richmond
1-223          1-2  richmond


### Concat Implementation

In [151]:
# merged data
big_df = pd.concat(valid_dfs, join="outer")
big_df

,local_code,locality
code_va,,
1-3.9,1-3,spotsylvania_county
1-13.3 et seq.,1-2,spotsylvania_county
2.1-116.1,16-147,spotsylvania_county
2.1-116.1 et seq.,16-147,spotsylvania_county
2.1-340 et seq.,2-107(b),spotsylvania_county
...,...,...
"59.1-142, 59.1-143 10-7","59.1-142, 59.1-143 10-7",colonial_beach
59.1-144,10-7,colonial_beach
NaN,10-8,colonial_beach


In [152]:
big_df.groupby("code_va").get_group("1-1")

,local_code,locality
code_va,,
1-1,1-2,falls_church
1-1,1-2,pulaski
1-1,1-2,clifton_forge
1-1,1-2,christiansburg


## Fips

In [153]:
circuit_courts_data = pd.read_csv('circuit_courts.csv').rename(columns={"fips":"circuit"})
circuit_courts_data

,circuit,name
0,1,Accomack Circuit Court
1,3,Albemarle Circuit Court
2,5,Alleghany Circuit Court
3,7,Amelia Circuit Court
4,9,Amherst Circuit Court
...,...,...
114,800,Suffolk Circuit Court
115,810,Virginia Beach Circuit Court
116,820,Waynesboro Circuit Court
117,830,Williamsburg/James City County Circuit Court


In [154]:
# clean name
# strip circuit court from name, make lower case, convert spaces to underscores
names = circuit_courts_data["name"]
for name in names:
    circuit_courts_data = circuit_courts_data.replace(name, name.replace(" Circuit Court", "").lower().replace(" ", "_"))
circuit_courts_data

,circuit,name
0,1,accomack
1,3,albemarle
2,5,alleghany
3,7,amelia
4,9,amherst
...,...,...
114,800,suffolk
115,810,virginia_beach
116,820,waynesboro
117,830,williamsburg/james_city_county


In [155]:
district_courts_data = pd.read_csv('district_courts.csv').rename(columns={"fips":"district"})
district_courts_data

,district,name
0,1,Accomack General District Court
1,3,Albemarle General District Court
2,5,Alleghany General District Court
3,7,Amelia General District Court
4,9,Amherst General District Court
...,...,...
128,800,Suffolk General District Court
129,810,Virginia Beach General District Court
130,820,Waynesboro General District Court
131,830,Williamsburg/James City County General Distric...


In [156]:
# clean name
# strip district court from name, make lower case, convert spaces to underscores
names = district_courts_data["name"]
for name in names:
    district_courts_data = district_courts_data.replace(name, name.replace(" General District Court", "").lower().replace(" ", "_"))
district_courts_data

,district,name
0,1,accomack
1,3,albemarle
2,5,alleghany
3,7,amelia
4,9,amherst
...,...,...
128,800,suffolk
129,810,virginia_beach
130,820,waynesboro
131,830,williamsburg/james_city_county


In [157]:
# merge district and circuit data 
fips = pd.merge(circuit_courts_data, district_courts_data, on="name", how="outer")
fips

,circuit,name,district
0,1.0,accomack,1.0
1,3.0,albemarle,3.0
2,5.0,alleghany,5.0
3,7.0,amelia,7.0
4,9.0,amherst,9.0
...,...,...,...
138,NaN,norfolk-civil,713.0
139,NaN,richmond-marsh_criminal/traffic_at_manchester,761.0
140,NaN,richmond-john_marshall_criminal/traffic,762.0
141,NaN,richmond-civil,763.0


In [158]:
# check to make sure all fips are present in result after merge
district_fips = list(district_courts_data["district"])
circuit_fips = list(circuit_courts_data["circuit"])

counter = 0
for fip in district_fips:
    if fip in list(fips["district"]):
        counter+=1
assert counter == len(district_fips)
print(f"counter: {counter} list: {len(district_fips)}")


counter: 133 list: 133


In [159]:
counter = 0
for fip in circuit_fips:
    if fip in list(fips["circuit"]):
        counter+=1
assert counter == len(circuit_fips)
print(f"counter: {counter} list: {len(circuit_fips)}")

counter: 119 list: 119


In [191]:
# rename name column 
fips = fips.rename(columns = {"name": "locality"})
fips

,circuit,locality,district
0,1.0,accomack,1.0
1,3.0,albemarle,3.0
2,5.0,alleghany,5.0
3,7.0,amelia,7.0
4,9.0,amherst,9.0
...,...,...,...
138,NaN,norfolk-civil,713.0
139,NaN,richmond-marsh_criminal/traffic_at_manchester,761.0
140,NaN,richmond-john_marshall_criminal/traffic,762.0
141,NaN,richmond-civil,763.0


The county df includes the word "county" in the locality name while the fips df does not. The next block updates the county df to remove county declaration and to only include the word "city" if the locality is a city. Maintaining the city declaration is important for localities like Fairfax that have the same name for both the county and city. This update will result in a cleaner merge becuase we are now seeing duplication due to the different naming conventions. 

In [195]:
# update county df
for locality in localities: 
    if "/" not in locality: 
        big_df = big_df.replace(locality, locality.replace("_county", ""))
big_df

,local_code,locality
code_va,,
1-3.9,1-3,spotsylvania
1-13.3 et seq.,1-2,spotsylvania
2.1-116.1,16-147,spotsylvania
2.1-116.1 et seq.,16-147,spotsylvania
2.1-340 et seq.,2-107(b),spotsylvania
...,...,...
"59.1-142, 59.1-143 10-7","59.1-142, 59.1-143 10-7",colonial_beach
59.1-144,10-7,colonial_beach
NaN,10-8,colonial_beach


In [196]:
# merge fips df with county df
result = pd.merge(big_df, fips, how="outer")
result

,local_code,locality,circuit,district
0,1-3,spotsylvania,177.0,177.0
1,1-2,spotsylvania,177.0,177.0
2,16-147,spotsylvania,177.0,177.0
3,16-147,spotsylvania,177.0,177.0
4,2-107(b),spotsylvania,177.0,177.0
...,...,...,...,...
37442,NaN,norfolk-civil,NaN,713.0
37443,NaN,richmond-marsh_criminal/traffic_at_manchester,NaN,761.0
37444,NaN,richmond-john_marshall_criminal/traffic,NaN,762.0
37445,NaN,richmond-civil,NaN,763.0


In [197]:
localities = list(result["locality"].unique())
localities.sort()
# localities